In [ ]:
import json
import pickle
from copy import deepcopy
import numpy as np
import pandas as pd

In [ ]:
ls era_*.json

In [ ]:
rosters = []
START_YEAR = 1960
for YEAR in range(START_YEAR,2020):
    rosters.append(json.load(open('rosters/auto_roster_{}.json'.format(YEAR))))


In [ ]:
base = deepcopy(rosters[-1])
df = pd.read_csv('avg_stats.csv')
y = df.iloc[:,-15:]
y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
new_team = [
            ('PHX','PHO'),
            ('BKN','BRK'),
            ('CHA','CHO'),
            ('NOP','NOH'),
            ('NOH','NOK'),
            ('BRK','NJN'),
            ('OKC','SEA'),
            ('MEM','VAN'),
            ('WAS','WSB'),
            ('SAC','KCK'),
            ('LAC','SDC'),
            ('UTA','NOJ'),
            ('SDC','BUF'),
            ('NJN','NYN'),
            ('NYN','NYA'),
            ('NYA','NJA'),
            ('GSW','SFW'),
            ('SAS','DLC'),
            ('DLC','TEX'),
            ('HOU','SDR'),
            ('CHA','CHH'),
            ('SAS','SAA'),
            ('SAA','TEX'),
            ('DEN','DNA'),
            ('DNA','DNR'),
            ('WSB','CAP'),
            ('CAP','BAL'),
            ('BAL','CHZ'),
            ('CHZ','CHP'),
            ('NOH','KEN'),
            ('MIN','SDS'),
            ('SDS','SDA'),
            ('MIA','FLO'),
            ('FLO','MMF'),
            ('MMF','MNM'),
            ('SFW','PHW'),
            ('LAL','MNL'),
            ('UTA','UTS'),
            ('UTS','LAS'),
            ('LAS','ANA'),
            ('CHH','SSL'),
            ('SSL','CAR'),
            ('CAR','HSM'),
            ('DET','FTW'),
            ('ATL','STL'),
            ('STL','MLH'),
            ('MLH','TRI'),
            ('PHI','SYR'),
            ('TOR','PTC'),
            ('PTC','PTP'),
            ('PTP','MNP'),
            ('VAN','MMS'),
            ('MMS','MMT'),
            ('MMT','MMP'),
            ('MMP','NOB'),
            ('ORL','VIR'),
            ('VIR','WSA'),
            ('WSA','OAK'),
            ('SAC','KCK'),
            ('KCK','KCO'),
            ('KCO','KCK'),
            ('KCK','CIN'),
            ('CIN','ROC'),
            ('CHP','BLB'),
            ('IND','INA'),
            ('INA','INO')]
st = set()
for t in new_team:
    st.add(t[0])
    st.add(t[1])
abbrev_to_tid = {_['abbrev']:_['tid'] for _ in base['teams']}

for nt,ot in new_team:
    try:
        abbrev_to_tid[ot] = abbrev_to_tid[nt]
    except:
        print(nt,ot)

In [ ]:
abbrev_to_tid

In [ ]:
CURRENT_YEAR = 2019
from collections import defaultdict
player_years = defaultdict(list)
nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])
nc2 = nc2 /nc2.sum()


for i,roster in enumerate(rosters):
    for player in roster['players']:
        name = player['firstName'] + ' ' + player['lastName']
        ovr = sum(player['ratings'][0][k]*nc2[y_keys.index(km)] for k,km in y_map.items())
        if player['tid'] != -1:
            player_years[name].append((ovr,i+START_YEAR,[_ for _ in roster['teams'] if _['tid'] == player['tid']],player))

In [ ]:
for player in player_years:
    player_years[player] = sorted(player_years[player],reverse=True)[0]

In [ ]:
player_years['Chris Paul']

In [ ]:
optimal_teams = defaultdict(list)

for name,player in player_years.items():
    try:
        optimal_teams[abbrev_to_tid[player_years[name][2][0]['abbrev']]].append((player[0],name))
    except IndexError:
        pass#print(name)
    except:
        raise

In [ ]:
for team in optimal_teams:
    optimal_teams[team] = sorted(optimal_teams[team],reverse=True)

In [ ]:
players = []
for team in optimal_teams:
    for p in optimal_teams[team][:12]:
        po, py, pt, pr = player = player_years[p[1]]
        pr['tid'] = team
        pr['born']['year'] = CURRENT_YEAR - (py-pr['born']['year'])
        players.append(pr)
    for p in optimal_teams[team][12:]:
        po, py, pt, pr = player = player_years[p[1]]
        pr['tid'] = -1
        pr['born']['year'] = CURRENT_YEAR - (py-pr['born']['year'])
        if po > 55:
            players.append(pr)

In [ ]:
base['players'] = players
with open('era_{}.json'.format(START_YEAR),'wt') as fp:
    json.dump(base,fp, sort_keys=True)
len(players)

In [ ]:
abbrev_to_tid[player_years['LeBron James'][2][0]['abbrev']]

In [ ]:
player_years['LeBron James']
base['gameAttributes']


In [ ]:
players

In [ ]:
player_years.keys()